In [6]:
from VFXTesting.lol_champ_reader import LoLChampReader
import dotenv
import pathlib
import random
import google.generativeai as genai


media = pathlib.Path("Data/Videos")

LOL_CHAMP_READER = LoLChampReader()
VIDEO_DIR = "Data/Videos"
SCHEMA_PATH = "Schemas/vfx_properties.json"

gemini_key = dotenv.get_key('.env', 'GEMINI_KEY')
if not gemini_key:
    raise ValueError("GEMINI_KEY environment variable not set.")
genai.configure(api_key=gemini_key)
MODEL = "gemini-1.5-pro-002"
model = genai.GenerativeModel(MODEL)

In [13]:
import json

def load_schema(schema_path=SCHEMA_PATH):
    with open(schema_path, 'r') as f:
        schema = json.load(f)
    return schema

In [12]:
import os

def get_random_videos(video_directory, sample_size=10):
    all_videos = [f for f in os.listdir(video_directory) if parse_video_filename(f) is not None]
    sample_videos = random.sample(all_videos, min(sample_size, len(all_videos)))
    return sample_videos


def parse_video_filename(filename):
    """
    Helper function to extract champion name and ability from the file name
    """
    base_name = os.path.splitext(filename)[0]
    try:
        split = base_name.split('_')
        champ_name = split[0]
        ability = split[1]
        if ability not in {'Q', 'W', 'E', 'R'}:
            return None
        return champ_name, ability
    except ValueError:
        return None

In [9]:
import time
from typing import Literal


def generate_ground_truth(schema, video_file, champ_name, ability: Literal['Q', 'W', 'E', 'R'] = 'Q'):
    """
    Generates ground truth data for a video using Gemini.
    """
    ability_info = LOL_CHAMP_READER.get_champ_ability(champ_name, ability)
    if not ability_info:
        return None

    ability_info_str = json.dumps(ability_info, indent=2)

    video = genai.upload_file(media / video_file)
    while video.state.name == "PROCESSING":
        print("processing video...")
        time.sleep(10)
        video = genai.get_file(video.name)

    prompt = [
        video,
        f"Given the contents of this video and the following JSON ability info: {ability_info_str}",
        f"Reply only with the key information using the JSON schema {schema}",
    ]

    response = model.generate_content(prompt)
    return response.text

In [20]:
schema = load_schema("Schemas/key_video_moments.json")
videos = get_random_videos(VIDEO_DIR, sample_size=1)

for video in videos:
    video = "Blitzcrank_E.webm" # TODO: REMOVE
    try:
        champ_name, ability = parse_video_filename(video)
        ground_truth = generate_ground_truth(schema, video, champ_name, ability)
        print(video, ground_truth)
    except ValueError as e:
        print(f"Skipping file {video}: {e}")

processing video...
Blitzcrank_E.webm ```json
{'key_video_moment': [{'startTime': 0.0, 'endTime': 0.1, 'vfx_description': 'Blitzcrank's right arm glows yellow and sparks emanate. He winds it up in preparation for an enhanced auto attack.', 'audio_description': ''}]}
```


### VFX Properties evaluation
Get a collection of 10 random videos and manually assess them.

### Hallucination Evaluation
1. Evaluate Gemini on itself in **key video moments schema** by cutting the videos into segments and seeing if Gemini can confirm the visual descriptions that it has itself given. **Hallucination evaluation**
2. Combine a video with an incorrect ability JSON and see if there are mentions of the incorrect ability or champion
3. Combine a video with anb incorrect ability JSON and ask if the two are different
4. Use the **timing schema** to generate start and stop times using Gemini, then compare to the "precise" sound timings
5. Color hallucination manual inspection with color picker for **vfx properties schema**
6. Reporting any other hallucinations seen along the way